In [1]:
import youtube_dl
import os
import sys
import shutil
from moviepy.editor import VideoFileClip, concatenate_videoclips
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

from cvcalib.audiosync import offset

curr_dir = "/home/sean/Documents/experiments/stage-mix-generator/"
download_dir = os.path.join(curr_dir, 'temp')

In [43]:
song_text = "Boy With Luv BTS"
lyrics_video_url = ""
stage_video_urls = []

In [44]:
textToSearch = song_text+' lyrics'

query = urllib.parse.quote(textToSearch)
url = "https://www.youtube.com/results?search_query=" + query
response = urllib.request.urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'}):
    if 'stage mix' not in vid['title'].lower():
        lyrics_video_url = 'https://www.youtube.com' + vid['href']
        break

In [45]:
textToSearch = song_text+' stage live'

query = urllib.parse.quote(textToSearch)
url = "https://www.youtube.com/results?search_query=" + query
response = urllib.request.urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'}):
    if 'stage mix' not in vid['title'].lower():
        stage_video_urls.append('https://www.youtube.com' + vid['href'])
        
        if len(stage_video_urls) >= 5:
            break

In [46]:
# download videos

ydl_opts = {
    'format': 'mp4',
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([lyrics_video_url])
    meta = ydl.extract_info(
        lyrics_video_url, download=False)
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, "lyrics_video.mp4"))
    
    for i, url in enumerate(stage_video_urls):
        ydl.download([url])
        meta = ydl.extract_info(
            url, download=False)
        
        shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, f"stage_video{i}.mp4"))

[youtube] k2lzg6-KfZ8: Downloading webpage
[youtube] k2lzg6-KfZ8: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/k2lzg6-KfZ8.mp4
[download] 100% of 7.56MiB in 00:0051MiB/s ETA 00:001
[youtube] k2lzg6-KfZ8: Downloading webpage
[youtube] k2lzg6-KfZ8: Downloading video info webpage
[youtube] D2xmlzMOsmM: Downloading webpage
[youtube] D2xmlzMOsmM: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/D2xmlzMOsmM.mp4
[download] 100% of 25.11MiB in 00:0127MiB/s ETA 00:00known ETA
[youtube] D2xmlzMOsmM: Downloading webpage
[youtube] D2xmlzMOsmM: Downloading video info webpage
[youtube] O4NB73HTlxI: Downloading webpage
[youtube] O4NB73HTlxI: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/O4NB73HTlxI.mp4
[download] 100% of 22.22MiB in 00:0173MiB/s ETA 00:00known ETA
[youtube] O4NB73HTlxI: Downloading webpag

In [41]:
lyrics_video_path = os.path.join(download_dir, 'lyrics_video.mp4')
stage_video_paths = [ os.path.join(download_dir, f'stage_video{i}.mp4') for i in range(len(stage_video_urls)) ]

# find timestamp of start of song in stage videos
offset.find_time_offset(['lyrics_video.mp4', 'stage_video0.mp4'], download_dir+'/', [0, 0])

((0.2322, 0), 29.97)

In [47]:
stage_song_starts = []
for i in range(len(stage_video_urls)):
    output = offset.find_time_offset(['lyrics_video.mp4', f'stage_video{i}.mp4'], download_dir+'/', [0, 0])
    print(output)
    stage_song_starts.append(output[0])

((0, 35.9097), 29.97)
((0, 9.555), 29.97)
((0, 3.8545), 29.97)
((0, 6.8034), 29.97)
((62.9261, 0), 29.97)


In [52]:
# get subclips of the song in the stage videos

# get length of lyrics video
lyrics_video = VideoFileClip(lyrics_video_path)

stage_videos = []
for i, path in enumerate(stage_video_paths):
    if stage_song_starts[i][0] == 0:
        stage_videos.append(VideoFileClip(path).subclip(stage_song_starts[i][1]))

In [59]:
# every 5 seconds during the song, splice clips from different performances

clips = []
stage_video_idx = 0
for step in np.arange(0, lyrics_video.duration, 5):
    if step+5 < stage_videos[stage_video_idx].duration:
        clips.append(stage_videos[stage_video_idx].subclip(step, step+5))
    
    stage_video_idx = (stage_video_idx + 1) % len(stage_videos)
    
final_clip = concatenate_videoclips(clips)

#final_clip = final_clip.set_duration(lyrics_video.duration)
#final_clip = final_clip.set_audio(lyrics_video.audio)

final_clip = final_clip.set_audio(stage_videos[0].audio)
final_clip.write_videofile("stage_mix.mp4")

                                                                       
                                                                    



                                                         




                                                 

                                                               


                                                            





chunk:  99%|█████████▉| 4805/4852 [2:09:24<00:00, 669.26it/s, now=None]



chunk:  97%|█████████▋| 4581/4741 [34:03<00:00, 844.90it/s, now=None]




t:  14%|█▍        | 959/6894 [20:38<00:44, 134.53it/s, now=None]

chunk:  99%|█████████▉| 4582/4631 [40:06<00:00, 892.74it/s, now=None]


chunk:  99%|█████████▉| 4582/4631 [38:30<00:00, 828.19it/s, now=None]





                                                                       A
                                                                    



                                                         




                                          

Moviepy - Building video stage_mix.mp4.
MoviePy - Writing audio in %s









chunk:   2%|▏         | 85/5550 [00:00<03:17, 27.66it/s, now=None]






chunk:   3%|▎         | 167/5550 [00:00<02:18, 38.94it/s, now=None]






chunk:   5%|▍         | 267/5550 [00:00<01:36, 54.71it/s, now=None]






chunk:   6%|▋         | 360/5550 [00:00<01:08, 76.23it/s, now=None]






chunk:   8%|▊         | 460/5550 [00:00<00:48, 105.41it/s, now=None]






chunk:  10%|█         | 558/5550 [00:00<00:34, 143.85it/s, now=None]






chunk:  12%|█▏        | 650/5550 [00:00<00:25, 192.50it/s, now=None]






chunk:  14%|█▎        | 750/5550 [00:00<00:18, 253.85it/s, now=None]






chunk:  15%|█▌        | 846/5550 [00:01<00:14, 325.46it/s, now=None]






chunk:  17%|█▋        | 946/5550 [00:01<00:11, 407.15it/s, now=None]






chunk:  19%|█▉        | 1045/5550 [00:01<00:09, 493.88it/s, now=None]






chunk:  21%|██        | 1142/5550 [00:01<00:07, 577.39it/s, now=None]






chunk:  22%|██▏       | 1238/5550 [00:01<00:06, 641.77it/s, now=None]






chunk:  24%|██▍     

MoviePy - Done.
Moviepy - Writing video stage_mix.mp4










t:   1%|          | 60/6894 [00:00<00:22, 305.52it/s, now=None]






t:   1%|          | 84/6894 [00:00<00:24, 281.93it/s, now=None]






t:   2%|▏         | 112/6894 [00:00<00:24, 278.93it/s, now=None]






t:   2%|▏         | 133/6894 [00:00<00:26, 252.39it/s, now=None]






t:   2%|▏         | 154/6894 [00:00<00:38, 174.02it/s, now=None]






t:   3%|▎         | 187/6894 [00:00<00:33, 202.59it/s, now=None]






t:   3%|▎         | 210/6894 [00:00<00:33, 201.78it/s, now=None]






t:   3%|▎         | 240/6894 [00:01<00:29, 222.74it/s, now=None]






t:   4%|▍         | 266/6894 [00:01<00:28, 229.63it/s, now=None]






t:   4%|▍         | 293/6894 [00:01<00:27, 240.16it/s, now=None]






t:   5%|▍         | 318/6894 [00:01<00:32, 201.13it/s, now=None]






t:   5%|▍         | 340/6894 [00:01<00:31, 205.82it/s, now=None]






t:   5%|▌         | 362/6894 [00:01<00:31, 208.70it/s, now=None]






t:   6%|▌         | 390/6894 [00:01<00:28, 224.54it/s, now=None]






t

t:  29%|██▉       | 2016/6894 [00:10<00:30, 162.23it/s, now=None]






t:  30%|██▉       | 2035/6894 [00:10<00:30, 159.56it/s, now=None]






t:  30%|██▉       | 2057/6894 [00:11<00:28, 170.51it/s, now=None]






t:  30%|███       | 2079/6894 [00:11<00:26, 182.55it/s, now=None]






t:  30%|███       | 2099/6894 [00:11<00:27, 176.43it/s, now=None]






t:  31%|███       | 2118/6894 [00:11<00:31, 152.66it/s, now=None]






t:  31%|███       | 2139/6894 [00:11<00:29, 162.37it/s, now=None]






t:  31%|███▏      | 2159/6894 [00:11<00:27, 171.69it/s, now=None]






t:  32%|███▏      | 2183/6894 [00:11<00:25, 184.20it/s, now=None]






t:  32%|███▏      | 2205/6894 [00:11<00:24, 188.35it/s, now=None]






t:  32%|███▏      | 2227/6894 [00:11<00:23, 196.07it/s, now=None]






t:  33%|███▎      | 2249/6894 [00:12<00:22, 201.99it/s, now=None]






t:  33%|███▎      | 2270/6894 [00:12<00:27, 165.78it/s, now=None]






t:  33%|███▎      | 2288/6894 [00:12<00:27, 164.87it/s, now=None

t:  57%|█████▋    | 3902/6894 [00:21<00:19, 153.31it/s, now=None]






t:  57%|█████▋    | 3930/6894 [00:21<00:16, 177.35it/s, now=None]






t:  57%|█████▋    | 3953/6894 [00:21<00:15, 189.72it/s, now=None]






t:  58%|█████▊    | 3976/6894 [00:21<00:14, 197.74it/s, now=None]






t:  58%|█████▊    | 3998/6894 [00:21<00:14, 201.00it/s, now=None]






t:  58%|█████▊    | 4020/6894 [00:21<00:14, 203.76it/s, now=None]






t:  59%|█████▊    | 4046/6894 [00:22<00:13, 217.81it/s, now=None]






t:  59%|█████▉    | 4069/6894 [00:22<00:15, 183.37it/s, now=None]






t:  59%|█████▉    | 4090/6894 [00:22<00:14, 188.72it/s, now=None]






t:  60%|█████▉    | 4112/6894 [00:22<00:14, 192.87it/s, now=None]






t:  60%|█████▉    | 4133/6894 [00:22<00:14, 196.35it/s, now=None]






t:  60%|██████    | 4154/6894 [00:22<00:13, 198.87it/s, now=None]






t:  61%|██████    | 4176/6894 [00:22<00:13, 204.20it/s, now=None]






t:  61%|██████    | 4197/6894 [00:22<00:13, 192.83it/s, now=None

t:  86%|████████▌ | 5926/6894 [00:31<00:05, 193.21it/s, now=None]






t:  86%|████████▋ | 5948/6894 [00:31<00:04, 193.29it/s, now=None]






t:  87%|████████▋ | 5969/6894 [00:32<00:04, 192.14it/s, now=None]






t:  87%|████████▋ | 5990/6894 [00:32<00:04, 190.80it/s, now=None]






t:  87%|████████▋ | 6010/6894 [00:32<00:05, 153.57it/s, now=None]






t:  88%|████████▊ | 6036/6894 [00:32<00:05, 171.06it/s, now=None]






t:  88%|████████▊ | 6057/6894 [00:32<00:04, 179.47it/s, now=None]






t:  88%|████████▊ | 6077/6894 [00:32<00:04, 181.93it/s, now=None]






t:  88%|████████▊ | 6101/6894 [00:32<00:04, 194.81it/s, now=None]






t:  89%|████████▉ | 6122/6894 [00:32<00:04, 187.89it/s, now=None]






t:  89%|████████▉ | 6142/6894 [00:32<00:03, 191.13it/s, now=None]






t:  89%|████████▉ | 6162/6894 [00:33<00:04, 163.65it/s, now=None]






t:  90%|████████▉ | 6180/6894 [00:33<00:04, 164.55it/s, now=None]






t:  90%|████████▉ | 6199/6894 [00:33<00:04, 169.06it/s, now=None

Moviepy - Done !
Moviepy - video ready stage_mix.mp4
